In [1]:
import pandas as pd
import numpy as np
import os

ncs_raw = pd.read_csv('output.csv', sep=';')
print(ncs_raw.shape)
ncs_raw.head()

(4301, 106)


,channel_id,video_id,title,duration,view_count,t_step,datapoint_0,datapoint_1,datapoint_2,datapoint_3,...,datapoint_90,datapoint_91,datapoint_92,datapoint_93,datapoint_94,datapoint_95,datapoint_96,datapoint_97,datapoint_98,datapoint_99
0,UC4wUSUO1aZ_NyibCqIjpt0g,uvwKupWhBeo,Cjbeards - Turn Out The Light | ♫ Copyright Fr...,177.0,78692,1780.0,0.0,0.087549,0.222166,0.300548,...,0.991263,1.000000,0.991981,0.982782,0.963623,0.879948,0.810000,0.793417,0.776090,0.773811
1,UC4wUSUO1aZ_NyibCqIjpt0g,zPzTFzS5LdM,GRÄF - Money | ♫ Copyright Free Music,164.0,77067,1650.0,0.0,0.096735,0.221913,0.280821,...,0.750712,0.737447,0.732588,0.723849,0.716424,0.711846,0.708859,0.704044,0.699875,0.650049
2,UC4wUSUO1aZ_NyibCqIjpt0g,6rGNP3Fp7lI,Hurshel - Like A Boss | ♫ Copyright Free Music,141.0,50633,1420.0,0.0,0.058521,0.137603,0.242036,...,0.794186,0.773019,0.753829,0.755486,0.759222,0.750470,0.754177,0.730603,0.695209,0.704453
3,UC4wUSUO1aZ_NyibCqIjpt0g,7MIzu5byjDg,Kevin Silvester - You and I (ft. Julie Zorrill...,231.0,302199,2310.0,0.0,0.144716,0.260439,0.326125,...,0.976438,0.965153,0.963948,0.957210,0.960841,0.962298,0.961726,0.964707,0.964861,0.983167
4,UC4wUSUO1aZ_NyibCqIjpt0g,IxCtGRUtyc8,Cjbeards - Refuse To Be Denied | ♫ Copyright F...,167.0,149326,1670.0,0.0,0.058395,0.151705,0.202970,...,0.627504,0.595213,0.569875,0.541354,0.509590,0.490040,0.482935,0.483111,0.490300,0.515135


In [2]:
ncs_raw = ncs_raw[ncs_raw['datapoint_0'].notna()]
print(ncs_raw.shape)
ncs_raw.head()

(4301, 106)


,channel_id,video_id,title,duration,view_count,t_step,datapoint_0,datapoint_1,datapoint_2,datapoint_3,...,datapoint_90,datapoint_91,datapoint_92,datapoint_93,datapoint_94,datapoint_95,datapoint_96,datapoint_97,datapoint_98,datapoint_99
0,UC4wUSUO1aZ_NyibCqIjpt0g,uvwKupWhBeo,Cjbeards - Turn Out The Light | ♫ Copyright Fr...,177.0,78692,1780.0,0.0,0.087549,0.222166,0.300548,...,0.991263,1.000000,0.991981,0.982782,0.963623,0.879948,0.810000,0.793417,0.776090,0.773811
1,UC4wUSUO1aZ_NyibCqIjpt0g,zPzTFzS5LdM,GRÄF - Money | ♫ Copyright Free Music,164.0,77067,1650.0,0.0,0.096735,0.221913,0.280821,...,0.750712,0.737447,0.732588,0.723849,0.716424,0.711846,0.708859,0.704044,0.699875,0.650049
2,UC4wUSUO1aZ_NyibCqIjpt0g,6rGNP3Fp7lI,Hurshel - Like A Boss | ♫ Copyright Free Music,141.0,50633,1420.0,0.0,0.058521,0.137603,0.242036,...,0.794186,0.773019,0.753829,0.755486,0.759222,0.750470,0.754177,0.730603,0.695209,0.704453
3,UC4wUSUO1aZ_NyibCqIjpt0g,7MIzu5byjDg,Kevin Silvester - You and I (ft. Julie Zorrill...,231.0,302199,2310.0,0.0,0.144716,0.260439,0.326125,...,0.976438,0.965153,0.963948,0.957210,0.960841,0.962298,0.961726,0.964707,0.964861,0.983167
4,UC4wUSUO1aZ_NyibCqIjpt0g,IxCtGRUtyc8,Cjbeards - Refuse To Be Denied | ♫ Copyright F...,167.0,149326,1670.0,0.0,0.058395,0.151705,0.202970,...,0.627504,0.595213,0.569875,0.541354,0.509590,0.490040,0.482935,0.483111,0.490300,0.515135


In [3]:
datapoints = ncs_raw[[f'datapoint_{i}' for i in range(100)]].values
print(datapoints.shape)
t_steps = ncs_raw['t_step'].values
print(t_steps.shape)

(4301, 100)
(4301,)


In [4]:
CUTOFF = 0.8
VID_LEN = datapoints.shape[0]
DP_LEN = datapoints.shape[1]

new_start = np.argmax((datapoints[:, 1:] - datapoints[:, :-1]) > 0, axis=1)
datapoints[np.arange(DP_LEN)[np.newaxis, :] < new_start[:, np.newaxis]] = 0
datapoints *= 1 / np.max(datapoints, axis=1)[:, np.newaxis]

peaks = np.argmax(datapoints, axis=1)

start_idxs = np.empty((VID_LEN))
end_idxs = np.empty((VID_LEN))

for i in range(VID_LEN):
    good_dp = datapoints[i] > CUTOFF
    prev_dp = ~good_dp[:peaks[i]][::-1]
    if prev_dp.any():
        start_idxs[i] = peaks[i] - np.argmax(prev_dp)
    else:
        start_idxs[i] = 0

    post_dp = ~good_dp[peaks[i]:]
    if post_dp.any():
        end_idxs[i] = peaks[i] + np.argmax(post_dp)
    else:
        end_idxs[i] = good_dp.shape[0]

start_s = (start_idxs * t_steps) // 1000
end_s = (end_idxs * t_steps) // 1000

print(start_s.shape)
print(end_s.shape)

(4301,)
(4301,)


In [5]:
ncs_raw['start_s'] = start_s
ncs_raw['end_s'] = end_s
ncs_raw.head()

,channel_id,video_id,title,duration,view_count,t_step,datapoint_0,datapoint_1,datapoint_2,datapoint_3,...,datapoint_92,datapoint_93,datapoint_94,datapoint_95,datapoint_96,datapoint_97,datapoint_98,datapoint_99,start_s,end_s
0,UC4wUSUO1aZ_NyibCqIjpt0g,uvwKupWhBeo,Cjbeards - Turn Out The Light | ♫ Copyright Fr...,177.0,78692,1780.0,0.0,0.087549,0.222166,0.300548,...,0.991981,0.982782,0.963623,0.879948,0.810000,0.793417,0.776090,0.773811,135.0,172.0
1,UC4wUSUO1aZ_NyibCqIjpt0g,zPzTFzS5LdM,GRÄF - Money | ♫ Copyright Free Music,164.0,77067,1650.0,0.0,0.096735,0.221913,0.280821,...,0.732588,0.723849,0.716424,0.711846,0.708859,0.704044,0.699875,0.650049,47.0,89.0
2,UC4wUSUO1aZ_NyibCqIjpt0g,6rGNP3Fp7lI,Hurshel - Like A Boss | ♫ Copyright Free Music,141.0,50633,1420.0,0.0,0.058521,0.137603,0.242036,...,0.753829,0.755486,0.759222,0.750470,0.754177,0.730603,0.695209,0.704453,82.0,127.0
3,UC4wUSUO1aZ_NyibCqIjpt0g,7MIzu5byjDg,Kevin Silvester - You and I (ft. Julie Zorrill...,231.0,302199,2310.0,0.0,0.144716,0.260439,0.326125,...,0.963948,0.957210,0.960841,0.962298,0.961726,0.964707,0.964861,0.983167,110.0,231.0
4,UC4wUSUO1aZ_NyibCqIjpt0g,IxCtGRUtyc8,Cjbeards - Refuse To Be Denied | ♫ Copyright F...,167.0,149326,1670.0,0.0,0.058395,0.151705,0.202970,...,0.569875,0.541354,0.509590,0.490040,0.482935,0.483111,0.490300,0.515135,113.0,136.0


In [6]:
print(ncs_raw.shape)
ncs_raw

(4301, 108)


,channel_id,video_id,title,duration,view_count,t_step,datapoint_0,datapoint_1,datapoint_2,datapoint_3,...,datapoint_92,datapoint_93,datapoint_94,datapoint_95,datapoint_96,datapoint_97,datapoint_98,datapoint_99,start_s,end_s
0,UC4wUSUO1aZ_NyibCqIjpt0g,uvwKupWhBeo,Cjbeards - Turn Out The Light | ♫ Copyright Fr...,177.0,78692,1780.0,0.000000,0.087549,0.222166,0.300548,...,0.991981,0.982782,0.963623,0.879948,0.810000,0.793417,0.776090,0.773811,135.0,172.0
1,UC4wUSUO1aZ_NyibCqIjpt0g,zPzTFzS5LdM,GRÄF - Money | ♫ Copyright Free Music,164.0,77067,1650.0,0.000000,0.096735,0.221913,0.280821,...,0.732588,0.723849,0.716424,0.711846,0.708859,0.704044,0.699875,0.650049,47.0,89.0
2,UC4wUSUO1aZ_NyibCqIjpt0g,6rGNP3Fp7lI,Hurshel - Like A Boss | ♫ Copyright Free Music,141.0,50633,1420.0,0.000000,0.058521,0.137603,0.242036,...,0.753829,0.755486,0.759222,0.750470,0.754177,0.730603,0.695209,0.704453,82.0,127.0
3,UC4wUSUO1aZ_NyibCqIjpt0g,7MIzu5byjDg,Kevin Silvester - You and I (ft. Julie Zorrill...,231.0,302199,2310.0,0.000000,0.144716,0.260439,0.326125,...,0.963948,0.957210,0.960841,0.962298,0.961726,0.964707,0.964861,0.983167,110.0,231.0
4,UC4wUSUO1aZ_NyibCqIjpt0g,IxCtGRUtyc8,Cjbeards - Refuse To Be Denied | ♫ Copyright F...,167.0,149326,1670.0,0.000000,0.058395,0.151705,0.202970,...,0.569875,0.541354,0.509590,0.490040,0.482935,0.483111,0.490300,0.515135,113.0,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4296,UCht8qITGkBvXKsR1Byln-wA,_8iypSHvdx0,The Story Unfolds – Jingle Punks (No Copyright...,90.0,910466,910.0,0.047479,0.000000,0.015310,0.039594,...,0.992905,0.965790,0.800172,0.605382,0.458813,0.347559,0.273205,0.215044,55.0,86.0
4297,UCht8qITGkBvXKsR1Byln-wA,duJ1dqpH0VU,Midsummer Night's Dream – Mendelssohn (No Copy...,302.0,127465,3030.0,0.000000,0.110487,0.324716,0.365525,...,0.490480,0.585952,0.697186,0.656235,0.535993,0.383659,0.268949,0.118740,184.0,190.0
4298,UCht8qITGkBvXKsR1Byln-wA,Sm0yTArTlHs,Retreat – Jason Farnham (No Copyright Music),109.0,147104,1090.0,0.031753,0.000000,0.045709,0.114506,...,0.979870,0.974214,0.928590,0.868375,0.773680,0.699513,0.658673,0.655484,71.0,104.0
4299,UCht8qITGkBvXKsR1Byln-wA,otAlmCnBgt0,Old Friend – Silent Partner (No Copyright Music),159.0,444133,1600.0,0.048818,0.000000,0.117514,0.208485,...,0.905984,0.862797,0.802935,0.699892,0.599818,0.506013,0.451630,0.423367,121.0,152.0


In [7]:
MAX_SONG_LEN = 240
ncs_raw.loc[ncs_raw['end_s'] > ncs_raw['duration'], 'end_s'] = ncs_raw[ncs_raw['end_s'] > ncs_raw['duration']]['duration']
ncs_raw = ncs_raw[ncs_raw['start_s'] < MAX_SONG_LEN]
ncs_raw.loc[ncs_raw['end_s'] > MAX_SONG_LEN, 'end_s'] = MAX_SONG_LEN
ncs_raw.shape

(4283, 108)

In [8]:
print(ncs_raw.shape)
ncs_raw.head()

(4283, 108)


,channel_id,video_id,title,duration,view_count,t_step,datapoint_0,datapoint_1,datapoint_2,datapoint_3,...,datapoint_92,datapoint_93,datapoint_94,datapoint_95,datapoint_96,datapoint_97,datapoint_98,datapoint_99,start_s,end_s
0,UC4wUSUO1aZ_NyibCqIjpt0g,uvwKupWhBeo,Cjbeards - Turn Out The Light | ♫ Copyright Fr...,177.0,78692,1780.0,0.0,0.087549,0.222166,0.300548,...,0.991981,0.982782,0.963623,0.879948,0.810000,0.793417,0.776090,0.773811,135.0,172.0
1,UC4wUSUO1aZ_NyibCqIjpt0g,zPzTFzS5LdM,GRÄF - Money | ♫ Copyright Free Music,164.0,77067,1650.0,0.0,0.096735,0.221913,0.280821,...,0.732588,0.723849,0.716424,0.711846,0.708859,0.704044,0.699875,0.650049,47.0,89.0
2,UC4wUSUO1aZ_NyibCqIjpt0g,6rGNP3Fp7lI,Hurshel - Like A Boss | ♫ Copyright Free Music,141.0,50633,1420.0,0.0,0.058521,0.137603,0.242036,...,0.753829,0.755486,0.759222,0.750470,0.754177,0.730603,0.695209,0.704453,82.0,127.0
3,UC4wUSUO1aZ_NyibCqIjpt0g,7MIzu5byjDg,Kevin Silvester - You and I (ft. Julie Zorrill...,231.0,302199,2310.0,0.0,0.144716,0.260439,0.326125,...,0.963948,0.957210,0.960841,0.962298,0.961726,0.964707,0.964861,0.983167,110.0,231.0
4,UC4wUSUO1aZ_NyibCqIjpt0g,IxCtGRUtyc8,Cjbeards - Refuse To Be Denied | ♫ Copyright F...,167.0,149326,1670.0,0.0,0.058395,0.151705,0.202970,...,0.569875,0.541354,0.509590,0.490040,0.482935,0.483111,0.490300,0.515135,113.0,136.0


In [9]:
ncs_raw.reset_index(drop=True, inplace=True)
print(ncs_raw.shape)
print(list(ncs_raw.index)[-10:])

(4283, 108)
[4273, 4274, 4275, 4276, 4277, 4278, 4279, 4280, 4281, 4282]


In [15]:
ncs_raw.to_csv('outgood.csv', sep=';')